In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.5/709.5 kB 12.5 MB/s eta 0:00:00


In [ ]:
!yolo track source="/content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4" save=True

WARNING ⚠️ 'model' is missing. Using default 'model=yolov8n.pt'.
Ultralytics YOLOv8.1.11 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (AMD EPYC 7B12)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

video 1/1 (1/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 3 persons, 127.0ms
video 1/1 (2/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 3 persons, 101.9ms
video 1/1 (3/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 3 persons, 97.9ms
video 1/1 (4/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 3 persons, 93.0ms
video 1/1 (5/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 4 persons, 92.1ms
video 1/1 (6/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 4 persons, 92.9ms
video 1/1 (7/114) /content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4: 640x384 4 persons, 91.7ms
video 1/1 (8/114) /content/WhatsApp V

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLO("yolov8n.pt")
names = model.model.names

video_path = "/content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("object_tracking.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLO("yolov8n.pt")
names = model.model.names

video_path = "/content/WhatsApp Video 2024-02-10 at 14.23.35_d5e34fec.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("object_tracking.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

people_count = 0  # Counter for the number of people detected

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Check if the track ID is new
                if track_id not in track_history:
                    people_count += 1
                    print(f"New person detected! Total people: {people_count}")

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()


New person detected! Total people: 1
New person detected! Total people: 2
New person detected! Total people: 3
New person detected! Total people: 4
New person detected! Total people: 5
New person detected! Total people: 6
New person detected! Total people: 7
New person detected! Total people: 8
New person detected! Total people: 9
